In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

Create some mock opening day payroll data for the desperately georgraphically confused "California Angels who play in a city called Anaheim, which is near a better know city called Los Angeles, which already has a baseball team"

In [2]:
payrollData = [['ANA', 2004, 1], ['LAA', 2005, 2], ['LAA', 2006, 3], ['LAA', 2007, float("NaN")]]
payrollDf = pd.DataFrame(payrollData, columns=['Team', 'Date', 'OpeningDayPayroll'])
payrollDf

,Team,Date,OpeningDayPayroll
0,ANA,2004,1.0
1,LAA,2005,2.0
2,LAA,2006,3.0
3,LAA,2007,NaN


The opening day payroll is valid for every day of a calendar year, so resample the dataset with forward fill to create a daily dataframe that can be matched to each game of a season.  

In [3]:
payrollDf['Date']=pd.to_datetime(payrollDf['Date'], format='%Y')
payrollDf.set_index('Date', inplace=True)
payrollDailyDf = payrollDf.resample('D').ffill()
payrollDailyDf.reset_index(inplace=True)
payrollDailyDf

,Date,Team,OpeningDayPayroll
0,2004-01-01,ANA,1.0
1,2004-01-02,ANA,1.0
2,2004-01-03,ANA,1.0
3,2004-01-04,ANA,1.0
4,2004-01-05,ANA,1.0
...,...,...,...
1092,2006-12-28,LAA,3.0
1093,2006-12-29,LAA,3.0
1094,2006-12-30,LAA,3.0
1095,2006-12-31,LAA,3.0


Create some (very) mock score data

In [4]:
scoreData = [['LAA', 'CLE', '2004-04-02', 1, 6, 2], ['NYY', 'LAA', '2005-08-03', 1, 2, 3], ['LAA', 'LAD', '2006-09-01', 1, 11, 2]]
scoreDataDf = pd.DataFrame(scoreData, columns=['HomeTeam', 'AwayTeam', 'Date', 'GameNo', 'HomeTeamScore', 'AwayTeamScore'])
scoreDataDf['Date']=pd.to_datetime(scoreDataDf['Date'], format='%Y-%m-%d')
scoreDataDf

,HomeTeam,AwayTeam,Date,GameNo,HomeTeamScore,AwayTeamScore
0,LAA,CLE,2004-04-02,1,6,2
1,NYY,LAA,2005-08-03,1,2,3
2,LAA,LAD,2006-09-01,1,11,2


Merge payroll and result data, expecting to find all games matched with an opening day payroll figure

In [5]:
mergedHome = pd.merge(payrollDailyDf, scoreDataDf, how='inner', left_on=['Date','Team'], right_on = ['Date','HomeTeam'])
mergedAway = pd.merge(payrollDailyDf, scoreDataDf, how='inner', left_on=['Date','Team'], right_on = ['Date','AwayTeam'])
mergedHome.append(mergedAway, ignore_index=True).sort_values(by=['Date','GameNo'])

,Date,Team,OpeningDayPayroll,HomeTeam,AwayTeam,GameNo,HomeTeamScore,AwayTeamScore
1,2005-08-03,LAA,2.0,NYY,LAA,1,2,3
0,2006-09-01,LAA,3.0,LAA,LAD,1,11,2


Outer join the data to identify which row wasn't matched.  The row without a match will be null.  Some special handling is required in this example because the data being matched could be in one of two columns (home or away)

In [6]:
mergedHome = pd.merge(payrollDailyDf, scoreDataDf.reset_index(), how='right', left_on=['Date','Team'], right_on = ['Date','HomeTeam'])
mergedAway = pd.merge(payrollDailyDf, scoreDataDf.reset_index(), how='right', left_on=['Date','Team'], right_on = ['Date','AwayTeam'])

mergedRight = mergedHome.append(mergedAway, ignore_index=True).sort_values(by=['Date','GameNo']).groupby(['index'])\
.agg({'Date':['max'], 'Team':['first'], 'OpeningDayPayroll':['max'], 'HomeTeam':['first'], 'HomeTeam':['first']})
mergedRight.columns = mergedRight.columns.get_level_values(0)
mergedRight[mergedRight.Team.isnull()]

,Date,Team,OpeningDayPayroll,HomeTeam
index,,,,
0,2004-04-02,None,NaN,LAA


Display the missed inner join data

In [7]:
payrollDailyDf[payrollDailyDf.Date.isin(mergedRight[mergedRight.Team.isnull()].Date)]

,Date,Team,OpeningDayPayroll
92,2004-04-02,ANA,1.0
